In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns

In [ ]:
# increase the nrows if your computer allows
data=pd.read_csv("train.csv",nrows=200000)
test=pd.read_csv("test.csv")
data

In [ ]:
data.shape

In [ ]:
data.corr()

In [ ]:
data.isna().any

In [ ]:
# visualizing data nan value
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
f, ax = plt.subplots(figsize=(15, 6))
plt.xticks(rotation='90')
sns.barplot(x=missing_data.index, y=missing_data['Percent'])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
missing_data.head()

In [ ]:
#data['key'][0][:4]
print(int(data['key'][0][11:13],10))
print(int(data['key'][0][14:16],10))

In [ ]:
pow(10,len(data['key'][0][14:16]))

In [ ]:
data['time']=data['key'].map(lambda dt: (int(dt[11:13],10))+ pow(10,-len(dt[14:16])) *  int(dt[14:16],10))
test['time']=test['key'].map(lambda dt: (int(dt[11:13],10))+ pow(10,-len(dt[14:16])) *  int(dt[14:16],10))

In [ ]:
data['year']=data['key'].map(lambda dt: (int(dt[:4],10)) )
test['year']=test['key'].map(lambda dt: (int(dt[:4],10)) )

In [ ]:
data=data.drop(['key'], axis=1)
data=test.drop(['key'], axis=1)

In [ ]:
data

In [ ]:
data['pickup_time']=data['pickup_datetime'].map(lambda dt: (int(dt[11:13],10))+ pow(10,-len(dt[14:16])) *  int(dt[14:16],10))

In [ ]:
data['pickup_year']=data['pickup_datetime'].map(lambda dt: (int(dt[:4],10)) )

In [ ]:
data['pickup_month']=data['pickup_datetime'].map(lambda dt: (int(dt[5:7],10)) )

In [ ]:
data=data.drop(['pickup_datetime'], axis=1)

In [ ]:
data

In [ ]:
data['pickup_month'].plot.hist(alpha=0.7,bins=27);

In [ ]:
# visualizing data nan value
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
f, ax = plt.subplots(figsize=(15, 6))
plt.xticks(rotation='90')
sns.barplot(x=missing_data.index, y=missing_data['Percent'])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
missing_data.head()

In [ ]:
# replacing all nan value with median
for col in ['dropoff_latitude','dropoff_longitude']:
    data[col]=data[col].fillna(data[col].median())

In [ ]:
# pre processind data
lst=[]
for i in range(data.shape[0]):
    if(data['fare_amount'][i]<2.5):
        lst.append(i)
    elif(data['passenger_count'][i]<=0):
        lst.append(i)
    elif(data['dropoff_latitude'][i]>46 and data['dropoff_latitude'][i]<36):
        lst.append(i)
    elif(data['dropoff_longitude'][i]>-70 and data['dropoff_longitude'][i]<-80):
        lst.append(i)
    
    

In [ ]:
data=data.drop(lst)

In [ ]:
data.shape

In [ ]:
# plot b/w price & passenger
plt.scatter([x for x in data['fare_amount']], [y for y in data['passenger_count']])
plt.show()

In [ ]:
# plot b/w price & passenger
plt.scatter([x for x in data['fare_amount']], [y for y in data['pickup_month']])
plt.show()

In [ ]:
# plot b/w price & passenger
plt.scatter([x for x in data['fare_amount']], [y for y in data['pickup_year']])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,1:],data['fare_amount'],test_size=0.2,random_state=42)

In [ ]:

X_train

In [ ]:
# scale data for good results on PCA
from sklearn.preprocessing import StandardScaler
X_sca = StandardScaler()
X_train = X_sca.fit_transform(X_train)
X_test = X_sca.fit_transform(X_test)



In [ ]:
# baseline model 
Median=y_train.median(axis = 0, skipna = True) 

In [ ]:
basemodel_rmse=((y_train-Median)**2).mean(axis=0,skipna=True)

In [ ]:
basemodel_rmse

# FEATURE SELECTION USING VARIANCE

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)

In [ ]:
pca.fit_transform(X_train)
pca.fit_transform(X_test)

In [ ]:
pca.explained_variance_ratio_

# LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
y_pred = linreg.predict(X_test)

In [ ]:
from sklearn import metrics
import numpy as np
np.sqrt(metrics.mean_squared_error(y_test, y_pred))

# RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model= RandomForestClassifier()
model

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn import metrics
import numpy as np
np.sqrt(metrics.mean_squared_error(y_test, predictions))

# BOOSTING

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
np.sqrt(metrics.mean_squared_error(y_test, predictions))